In [1]:
import tensorflow as tf
import numpy as np

from tensorflow.keras import Sequential
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from sklearn.model_selection import train_test_split

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
%load_ext tensorboard

In [4]:
logdir = "logs\\hparam_tuning\\"

In [5]:
data = np.load("data.npy")
labels = np.load("labels.npy")
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.15, random_state=28)

y_train = tf.one_hot(y_train, 60)
y_test = tf.one_hot(y_test, 60)

In [17]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32]))
HP_CONV_KERNEL_SIZE = hp.HParam("conv_kernel_size", hp.Discrete([2, 3]))
HP_POOL_STRIDE_SIZE = hp.HParam("pool_stride_size", hp.Discrete([2, 3]))
HP_POOL_WINDOW_SIZE = hp.HParam("pool_window_size", hp.Discrete([2, 3]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.3))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.RealInterval(0.00001, 0.001))
HP_DENSE_SIZE = hp.HParam("dense_size", hp.Discrete([100, 500]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
HP_EPOCH_SIZE = hp.HParam("epoch_size", hp.Discrete([50, 400]))
HP_BATCH_SIZE = hp.HParam("batch_sizes", hp.Discrete([16, 64]))

CATEGORICAL_ACCURACY = "categorical_accuracy"

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_CONV_KERNEL_SIZE, HP_POOL_STRIDE_SIZE, HP_POOL_WINDOW_SIZE, HP_DROPOUT, HP_DENSE_SIZE, HP_OPTIMIZER, HP_EPOCH_SIZE, HP_BATCH_SIZE],
    metrics=[hp.Metric(CATEGORICAL_ACCURACY, display_name='CategoricalAccuracy')],
  )

In [18]:
def train_test_model(hparams):
  model = Sequential(
    [
        Conv2D(hparams[HP_NUM_UNITS], int(hparams[HP_CONV_KERNEL_SIZE]), padding="same", activation="relu", input_shape=(60, 173, 2)),
        MaxPooling2D(int(hparams[HP_POOL_WINDOW_SIZE]), strides= int(hparams[HP_POOL_STRIDE_SIZE])),
        Conv2D(32, int(hparams[HP_CONV_KERNEL_SIZE]), padding="same", activation="relu"),
        MaxPooling2D(int(hparams[HP_POOL_WINDOW_SIZE]), strides= int(hparams[HP_POOL_STRIDE_SIZE])),
        Conv2D(64, int(hparams[HP_CONV_KERNEL_SIZE]), padding="same", activation="relu"),
        MaxPooling2D(int(hparams[HP_POOL_WINDOW_SIZE]), strides= int(hparams[HP_POOL_STRIDE_SIZE])),
        Flatten(),
        Dense(hparams[HP_DENSE_SIZE], activation="relu"),
        Dropout(hparams[HP_DROPOUT]),
        Dense(60, activation="softmax")
    ]
  )
  model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='categorical_crossentropy',
      metrics=['categorical_accuracy'],
  )

  model.optimizer.lr.assign(hparams[HP_LEARNING_RATE])

  model.fit(x_train, y_train,
    batch_size=int(hparams[HP_BATCH_SIZE]),
    epochs=int(hparams[HP_EPOCH_SIZE]),
    callbacks=[
        tf.keras.callbacks.TensorBoard(logdir),  # log metrics
        hp.KerasCallback(logdir, hparams),  # log hparams
        # tf.keras.callbacks.EarlyStopping("val_loss", patience=20)
    ],
    validation_split=0.1,
    verbose=2
  )
  _, accuracy = model.evaluate(x_test, y_test)
  return accuracy, model

In [19]:
session_num = 0
models = dict()

for num_units in HP_NUM_UNITS.domain.values:
    for optimizer in HP_OPTIMIZER.domain.values:
        for kernel_size in HP_CONV_KERNEL_SIZE.domain.values:
            for stride_size in HP_POOL_STRIDE_SIZE.domain.values:
                for window_size in HP_POOL_WINDOW_SIZE.domain.values:
                    for learning_rate in np.linspace(HP_LEARNING_RATE.domain.min_value, HP_LEARNING_RATE.domain.max_value, 4):
                        for epoch_size in np.linspace(HP_EPOCH_SIZE.domain.values[0], HP_EPOCH_SIZE.domain.values[1], 4):
                            for batch_size in np.linspace(HP_BATCH_SIZE.domain.values[0], HP_BATCH_SIZE.domain.values[1], 4):
                                for dropout_rate in np.linspace(HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value, 2):
                                    for dense_size in np.linspace(HP_DENSE_SIZE.domain.values[0], HP_DENSE_SIZE.domain.values[1], 5):
                                        hparams = {
                                            HP_NUM_UNITS: num_units,
                                            HP_DROPOUT: dropout_rate,
                                            HP_OPTIMIZER: optimizer,
                                            HP_EPOCH_SIZE: epoch_size,
                                            HP_BATCH_SIZE: batch_size,
                                            HP_CONV_KERNEL_SIZE: kernel_size,
                                            HP_POOL_STRIDE_SIZE: stride_size,
                                            HP_POOL_WINDOW_SIZE: window_size,
                                            HP_DENSE_SIZE: dense_size,
                                            HP_LEARNING_RATE: learning_rate,
                                        }
                                        
                                        run_name = "run-%d" % session_num
                                        print('--- Starting trial: %s' % run_name)
                                        print({h.name: hparams[h] for h in hparams})
                                        _, model = train_test_model(hparams)
                                        models[session_num] = model
                                        session_num += 1
    #                                     break
    #                                 break
    #                             break
    #                         break
    #                     break
    #                 break
    #             break
    #         break
    #     break
    # break


--- Starting trial: run-0
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'adam', 'epoch_size': 50.0, 'batch_sizes': 16.0, 'conv_kernel_size': 2, 'pool_stride_size': 2, 'pool_window_size': 2, 'dense_size': 100.0, 'learning_rate': 1e-05}
Epoch 1/50
27/27 - 8s - loss: 54.9480 - categorical_accuracy: 0.0188 - val_loss: 23.0426 - val_categorical_accuracy: 0.0417
Epoch 2/50
27/27 - 0s - loss: 26.8696 - categorical_accuracy: 0.0235 - val_loss: 13.4638 - val_categorical_accuracy: 0.0208
Epoch 3/50
27/27 - 0s - loss: 16.4372 - categorical_accuracy: 0.0329 - val_loss: 8.9772 - val_categorical_accuracy: 0.0208
Epoch 4/50
27/27 - 0s - loss: 10.1444 - categorical_accuracy: 0.0141 - val_loss: 6.7257 - val_categorical_accuracy: 0.0417
Epoch 5/50
27/27 - 0s - loss: 6.9512 - categorical_accuracy: 0.0329 - val_loss: 5.6097 - val_categorical_accuracy: 0.0208
Epoch 6/50
27/27 - 0s - loss: 6.0294 - categorical_accuracy: 0.0259 - val_loss: 5.0528 - val_categorical_accuracy: 0.0208
Epoch 7/50
27/27 - 0s - lo